In [ ]:
import pandas as pd
import numpy as np
import sklearn
import scikeras
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

**Carregamento das Bases**

In [4]:
X_treinamento = pd.read_csv("/content/train.csv")
X_teste = pd.read_csv("/content/test.csv") #Realizar a previsão final
y = pd.read_csv("/content/gender_submission.csv") #validação

**Base de Treinamento**

In [18]:
X_treinamento_new = X_treinamento
#X_treinamento

y_treinamento = X_treinamento['Survived']
#y_treinamento = X_treinamento['PassengerId','Survived']

# Remoção de colunas
colunas_para_remover = ["Name", "Ticket", "Cabin", "PassengerId", "Survived", "Fare"]
X_treinamento_new = X_treinamento_new.drop(colunas_para_remover, axis=1)

#Tratamento variáveis categóricas
le = LabelEncoder()
X_treinamento_new['Sex'] = le.fit_transform(X_treinamento_new['Sex'])
X_treinamento_new['Embarked'] = le.fit_transform(X_treinamento_new['Embarked'])

#Tratamento de valores ausentes (Age)
mediana_idade = X_treinamento_new['Age'].median()
X_treinamento_new['Age'] = X_treinamento_new['Age'].fillna(mediana_idade)

print("Valores ausentes após o pré-processamento:")
print(X_treinamento_new.isnull().sum())

Valores ausentes após o pré-processamento:
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64


**Hiperparâmetros**

In [38]:
modelo_rf_cv = RandomForestClassifier(random_state=42)
parametros = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, None],  # None significa sem limite
    'min_samples_split': [2, 5, 10]
}

# Usando GridSearchCV para encontrar a melhor combinação
grid_search = GridSearchCV(estimator=modelo_rf_cv, param_grid=parametros, cv=5, scoring='accuracy')

# Treinando o grid search
grid_search.fit(X_treinamento_new, y_treinamento)

# Mostrando os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:", grid_search.best_params_)
print("Melhor acurácia (média da validação cruzada):", grid_search.best_score_)
modelo_otimizado = grid_search.best_estimator_

Melhores parâmetros encontrados: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 200}
Melhor acurácia (média da validação cruzada): 0.8238026489234824


**Validação Cruzada**

In [39]:
scores = cross_val_score(modelo_otimizado, X_treinamento_new, y_treinamento, cv=5, scoring='accuracy')

print("Acurácias da validação cruzada:", scores)
print(f"Acurácia média da validação cruzada: {scores.mean():.2f}")

Acurácias da validação cruzada: [0.81564246 0.8258427  0.8258427  0.79213483 0.85955056]
Acurácia média da validação cruzada: 0.82


**Base de dados de Teste - Previsão**

In [44]:
X_teste_new = X_teste
#X_treinamento

passengerids = X_teste['PassengerId']
# Remoção de colunas que afetam o treinamento
colunas_para_remover = ["Name", "Ticket", "Cabin", "PassengerId", "Fare"]
X_teste_new = X_teste_new.drop(colunas_para_remover, axis=1)

#Tratamento variáveis categóricas
le = LabelEncoder()
X_teste_new['Sex'] = le.fit_transform(X_teste_new['Sex'])
X_teste_new['Embarked'] = le.fit_transform(X_teste_new['Embarked'])

#Tratamento de valores ausentes (Age)
mediana_idade = X_teste_new['Age'].median()
X_teste_new['Age'] = X_teste_new['Age'].fillna(mediana_idade)

print("Valores ausentes após o pré-processamento:")
print(X_teste_new.isnull().sum())

Valores ausentes após o pré-processamento:
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64


**Previsões**

In [45]:
y_teste = y['Survived']
previsoes = modelo_otimizado.predict(X_teste_new)
acuracia = accuracy_score(y_teste, previsoes)
print(f"Acurácia do modelo no conjunto de treinamento: {acuracia:.2f}")

Acurácia do modelo no conjunto de treinamento: 0.93


**Criação do arquivo CSV**

In [57]:
previsoes_int = previsoes.astype(int)
submissao = pd.DataFrame({
    'PassengerId':passengerids,
    'Survived': previsoes_int
})
submissao = submissao.set_index('PassengerId')

submissao.to_csv('Predict_Random_Forest.csv')